In [3]:
%pylab
%matplotlib inline
from PIL import Image
import numpy as np

Using matplotlib backend: QtAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [88]:
import cv2
img = cv2.imread("../data/Yeast_Cells.jpg")
rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
labImg = cv2.cvtColor(rgbImg, cv2.COLOR_RGB2LAB)

cv2.imshow('LAB image', labImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [91]:
# Python3 program change RGB Color
# Model to LAB Color Model

def rgb_to_lab(r, g, b):

# l, a, b = lightness, green-red, yellow-blue
# Conversion via CIE 1931-color model (XYZ-coordinates)
    X = 0.4124564 * r + 0.3575761 * g + 0.1804375 * b
    Y = 0.2126729 * r + 0.7151522 * g + 0.0721750 * b
    Z = 0.0193339 * r + 0.1191920 * g + 0.9503041 * b

# Import Numpy
    import numpy as np

# Convert XYZ to LAB
    L = 116 * np.power(Y/100, 1/3) - 16
    A = 500 * (np.power(X/95.047, 1/3) - np.power(Y/100, 1/3))
    B = 200 * (np.power(Y/100, 1/3) - np.power(Z/108.883, 1/3))

    return L, A, B

In [92]:
def LABColor(img):
    if isinstance(img, Image.Image):
        r,g,b = img.split()
        Ldat = []
        Adat = []
        Bdat = []
        for rd, gn, bl in zip(r.getdata(), g.getdata(), b.getdata()):
            L,A,B = rgb_to_lab(rd/255., gn/255., bl/255.)
            Ldat.append(int(L*255.))
            Adat.append(int(A*255.))
            Bdat.append(int(B*255.))
        r.putdata(Ldat)
        g.putdata(Adat)
        b.putdata(Bdat)
        return Image.merge("RGB", (r,g,b))
    else:
        return None


In [94]:
a = Image.open("../data/Cell_Nuclei.jpg")
b = LABColor(a)
b.show()